# Transfer Learning Example on Radar Dataset (Central Kansas)

## Importing the required modules

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(suppress=True)
import scipy.io as sio
import pandas as pd
import os
import re
import math
import itertools
import time
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import KFold
from itertools import product
from sklearn.utils.class_weight import compute_class_weight
torch.cuda.empty_cache()

# Reading the labels and creating train and test data

In [2]:
def index(string):
    s = re.findall("[0-9]", string)
    return int(''.join(s))

scenario_idx = 29 # Bonneville Salt Flats
names = os.listdir(f'num{scenario_idx}_NAMF_DATA_20_rng30_pulse1_1000_100k/')
names = sorted(names, key=index)
print(len(names))
y = pd.read_csv(f'num{scenario_idx}_Ground_Truth_20_rng30_1000_100k_m.csv')
col_names = y.columns[4:7]
y = y[col_names].to_numpy()

scenario_idx_test = 60 # Central Kansas
names_test = os.listdir(f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/')
names_test = sorted(names_test, key=index)
y_test = pd.read_csv(f'num{scenario_idx_test}_Ground_Truth_20_rng30_1000_100k_m.csv')
col_names_test = y_test.columns[4:7]
y_test = y_test[col_names_test].to_numpy()

# Define fine-tuning name and labels
few_shot_count = 64
names_FS = names_test[:few_shot_count]
y_FS = y_test[:few_shot_count]

# Create training and testing datasets
y_train = y[:int(0.9 * len(names))]
y_test = y_test[int(0.9 * len(names_test)) - 1:]
training_names = names[:int(0.9 * len(names))]
test_names = names_test[int(0.9 * len(names_test)) - 1:]

print('Training labels:')
print(y_train)

# Tensor Corners
##################################################################################
# num29: [10851, 215, -5.4], num60: [11073, 215, -5.3], num62: [11471, 215, -5.6]
# num76: [11388, 215, -6.15], num35: [11381, 215, -0.95]
##################################################################################

# Training dataset global constants
coord_tr = [10851, 215, -5.4]  # Tensor corner
rng_res_tr = 59.9585 / 2        # Range resolution
az_step_tr = 0.4                # Azimuth step size
el_step_tr = 0.01               # Elevation step size

# Test dataset global constants
coord_ts = [11073, 215, -5.3]  # Tensor corner
rng_res_ts = 59.9585 / 2        # Range resolution
az_step_ts = 0.4                # Azimuth step size
el_step_ts = 0.01               # Elevation step size

def Drawing_Batch(names, label, bs, ind, directory, normalize=True):
    x = []
    labels = []
    
    for j in range(ind * bs, (ind + 1) * bs):
        try:
            temp = sio.loadmat(directory + names[j])['P']
        except:
            break
        if normalize:
            Anorm = temp - np.min(temp.flatten())
            temp = np.divide(Anorm, np.max(Anorm.flatten()))
        x.append(temp)
        labels.append(label[j, :])
        
    x = torch.FloatTensor(np.array(x))
    labels = torch.FloatTensor(np.array(labels))
    return x, labels

100000
Training labels:
[[ 8.3355 13.325  19.525 ]
 [18.156  17.388  32.775 ]
 [ 8.777  24.73   18.582 ]
 ...
 [15.785  15.992  29.641 ]
 [ 3.1736 10.716  12.27  ]
 [ 4.5552 21.035  14.234 ]]


In [3]:
print(len(y_train))
print(len(y_test))

90000
10001


## Define a regression CNN and instantiating it

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(21, 32, 3, 1)
        self.conv2 = nn.Conv1d(32, 64, 3, 1)
        self.batchnorm1 = nn.BatchNorm1d(32)
        self.batchnorm2 = nn.BatchNorm1d(64)
        self.fc1 = nn.Linear(64 * 5, 20)  # Adjust input size based on the output of conv layers and max pooling
        self.fc2_reg = nn.Linear(20, 2)  # Adjusted output size

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.batchnorm1(x))
        x = F.max_pool1d(x, 2)
        
        x = self.conv2(x)
        x = F.relu(self.batchnorm2(x))
        x = F.max_pool1d(x, 2)
        
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        output_reg = self.fc2_reg(x)  # (bs, 2)
        
        return output_reg
    
from torchsummary import summary
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = Net()
model = model.to(device)
if device == 'cuda:0':
    model = torch.nn.DataParallel(model)
    cudnn.benchmark = True
print(summary(model,(21,26)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 32, 24]           2,048
       BatchNorm1d-2               [-1, 32, 24]              64
            Conv1d-3               [-1, 64, 10]           6,208
       BatchNorm1d-4               [-1, 64, 10]             128
            Linear-5                   [-1, 20]           6,420
            Linear-6                    [-1, 2]              42
               Net-7                    [-1, 2]               0
Total params: 14,910
Trainable params: 14,910
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 0.06
Estimated Total Size (MB): 0.08
----------------------------------------------------------------
None


## Global Definitions

In [5]:
bs = 128  # batch_size
num_epoch = 50  # number of epochs
PATH = './ckpt_model.pth'   # forsaving the model
criterion = nn.MSELoss()
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Define a Loss function and optimizer; Using GPU or CPU
model = Net()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
model = model.to(device)
if device == 'cuda:0':
    model = torch.nn.DataParallel(model)
    cudnn.benchmark = True
    
def Spher2Cart_1D(spherical):
    cartesian = np.zeros(3)
    hypotenuse = np.cos(np.radians(spherical[2]))*spherical[0]
    cartesian[0] = np.cos(np.radians(spherical[1]))*hypotenuse
    cartesian[1] = -np.sin(np.radians(spherical[1]))*hypotenuse
    cartesian[2] = np.sin(np.radians(spherical[2]))*spherical[0]
    return cartesian

## Train and evaluate the network

In [6]:
def main(training_names, test_names, bs, num_epoch, y_train, y_test, normalize=True):
    best_error = 1e+20  # a dummy and very large number for saving the best discovered model
    for epoch in range(num_epoch):
        print('Epoch {}/{}'.format(epoch, num_epoch))
        print('-' * 10)
        running_loss_train = 0
        running_loss_test = 0

        model.train()
        for i in range(0, len(training_names) // bs):
            x_train, labels = Drawing_Batch(training_names, y_train, bs, i, f'num{scenario_idx}_NAMF_DATA_20_rng30_pulse1_1000_100k/', normalize)
            x_train = x_train.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            out = model(x_train)
            loss = criterion(out, labels[:, 0:2])
            loss.backward()
            optimizer.step()
            running_loss_train += loss.item()
            
        out = torch.cat((out, torch.unsqueeze(labels[:,2], dim=1)), dim=1)

        # Training error display
        true_train = Spher2Cart_1D(np.multiply(labels.cpu().data.numpy()[1], [rng_res_tr, az_step_tr, el_step_tr]) + coord_tr)
        pred_train = Spher2Cart_1D(np.multiply(out.cpu().data.numpy()[1], [rng_res_tr, az_step_tr, el_step_tr]) + coord_tr)
        print(true_train)
        print(pred_train)

        model.eval()
        with torch.no_grad():
            for i in range(0, len(test_names) // bs):
                x_test, labels_test = Drawing_Batch(test_names, y_test, bs, i, f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/', normalize)
                x_test = x_test.to(device)
                labels_test = labels_test.to(device)
                out_test = model(x_test)
                loss_test = criterion(out_test, labels_test[:, 0:2])
                running_loss_test += loss_test.item()
                
        out_test = torch.cat((out_test, torch.unsqueeze(labels_test[:,2], dim=1)), dim=1)

        # Test error display
        true_test = Spher2Cart_1D(np.multiply(labels_test.cpu().data.numpy()[1], [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
        pred_test = Spher2Cart_1D(np.multiply(out_test.cpu().data.numpy()[1], [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
        print(true_test)
        print(pred_test)

        epoch_loss_train = running_loss_train * x_train.size(0) / len(training_names)
        epoch_loss_test = running_loss_test * x_test.size(0) / len(test_names)

        print('Train Loss: {:.6f} ---- Test Loss: {:.6f}'.format(epoch_loss_train, epoch_loss_test))
        if epoch % 5 == 0:
            if epoch_loss_test < best_error:
                torch.save(model.state_dict(), PATH)
                best_error = epoch_loss_test

start = time.time()
main(training_names, test_names, bs, num_epoch, y_train, y_test, normalize=True)
end = time.time()
print(end - start)

Epoch 0/50
----------
[-8187.45955885  7784.83170223 -1008.20172776]
[-8208.98872915  7739.2576875  -1006.80069973]
[-8409.5341829   7694.47736292 -1005.10206359]
[-8556.12846063  7498.93931165 -1003.22889385]
Train Loss: 8.054857 ---- Test Loss: 10.554716
Epoch 1/50
----------
[-8187.45955885  7784.83170223 -1008.20172776]
[-8199.77093405  7751.82279998 -1006.97230854]
[-8409.5341829   7694.47736292 -1005.10206359]
[-8505.14223197  7553.80061218 -1003.05807124]
Train Loss: 0.201115 ---- Test Loss: 6.945337
Epoch 2/50
----------
[-8187.45955885  7784.83170223 -1008.20172776]
[-8191.18866077  7763.90638139 -1007.15740655]
[-8409.5341829   7694.47736292 -1005.10206359]
[-8489.65447016  7574.3247008  -1003.2413048 ]
Train Loss: 0.131613 ---- Test Loss: 5.670177
Epoch 3/50
----------
[-8187.45955885  7784.83170223 -1008.20172776]
[-8196.30833711  7756.63212896 -1007.04274062]
[-8409.5341829   7694.47736292 -1005.10206359]
[-8487.0403089   7573.82132881 -1003.03975617]
Train Loss: 0.108559 


KeyboardInterrupt



In [7]:
def Spher2Cart_2D(spherical):
    cartesian = np.zeros((len(spherical), 3))
    hypotenuse = np.multiply(np.cos(np.radians(spherical[:, 2])), spherical[:, 0])
    cartesian[:, 0] = np.multiply(np.cos(np.radians(spherical[:, 1])), hypotenuse)
    cartesian[:, 1] = np.multiply(-np.sin(np.radians(spherical[:, 1])), hypotenuse)
    cartesian[:, 2] = np.multiply(np.sin(np.radians(spherical[:, 2])), spherical[:, 0])
    return cartesian

# Testing: (range, az, el)
model.eval()
out_test_reg = np.zeros((len(y_test), 3))
labels_test_reg = np.zeros((len(y_test), 3))

for i in range(0, len(y_test) // bs):
    x_test, labels_test = Drawing_Batch(test_names, y_test, bs, i, f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/', True)
    x_test = x_test.to(device)
    labels_test = labels_test.cpu().data.numpy()
    labels_test_reg[bs * i: bs * i + bs] = (labels_test[:, 0:3])

    cur_test_reg = model(x_test)
    out_test_reg[bs * i: bs * i + bs, 0:2] = cur_test_reg.cpu().data.numpy()
    out_test_reg[bs * i: bs * i + bs, 2] = labels_test_reg[bs * i: bs * i + bs, 2]

# Calculate azimuth estimation error
azim_tot = 0
for i in range(len(out_test_reg)):
    azim_tot += np.linalg.norm(out_test_reg[i, 1] - labels_test_reg[i, 1])

azim_tot = azim_tot / len(out_test_reg)
print(f'Azimuth Estimation Error (deg) = {azim_tot}')

# Convert spherical coordinates to Cartesian coordinates
new_data = Spher2Cart_2D(np.multiply(out_test_reg, [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
new_labels_data = Spher2Cart_2D(np.multiply(labels_test_reg, [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)

# Calculate localization error
sum_tot = 0
for i in range(0, len(new_data) - (len(new_data) % bs), 1):
    sum_tot += np.linalg.norm(new_data[i, :] - new_labels_data[i, :])

sum_tot = sum_tot / (len(new_data) - (len(new_data) % bs))
print(f'Localization Error (m) = {sum_tot}')

Azimuth Estimation Error (deg) = 1.0840089544512346
Localization Error (m) = 99.15726815648534


## Fine-Tuning Regression CNN

In [8]:
model.module.conv1.weight.requires_grad = False
model.module.conv1.bias.requires_grad = False
model.module.batchnorm1.weight.requires_grad = False
model.module.batchnorm1.bias.requires_grad = False

model.module.conv2.weight.requires_grad = False
model.module.conv2.bias.requires_grad = False
model.module.batchnorm2.weight.requires_grad = False
model.module.batchnorm2.bias.requires_grad = False

In [9]:
bs = 64  # batch_size
num_epoch = 100
optimizer = optim.Adam(model.parameters(), lr = 0.001)

def main(training_names, test_names, bs, num_epoch, y_train, y_test, normalize=True):
    best_error = 1e+20  # a dummy and very large number for saving the best discovered model
    for epoch in range(num_epoch):
        print('Epoch {}/{}'.format(epoch, num_epoch))
        print('-' * 10)
        running_loss_train = 0
        running_loss_test = 0

        model.train()
        for i in range(0, len(training_names) // bs):
            x_train, labels = Drawing_Batch(training_names, y_train, bs, i, f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/', normalize)
            x_train = x_train.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            out = model(x_train)
            loss = criterion(out, labels[:, 0:2])
            loss.backward()
            optimizer.step()
            running_loss_train += loss.item()
            
        out = torch.cat((out, torch.unsqueeze(labels[:,2], dim=1)), dim=1)

        # Training error display
        true_train = Spher2Cart_1D(np.multiply(labels.cpu().data.numpy()[1], [rng_res_tr, az_step_tr, el_step_tr]) + coord_tr)
        pred_train = Spher2Cart_1D(np.multiply(out.cpu().data.numpy()[1], [rng_res_tr, az_step_tr, el_step_tr]) + coord_tr)
        print(true_train)
        print(pred_train)

        model.eval()
        with torch.no_grad():
            for i in range(0, len(test_names) // bs):
                x_test, labels_test = Drawing_Batch(test_names, y_test, bs, i, f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/', normalize)
                x_test = x_test.to(device)
                labels_test = labels_test.to(device)
                out_test = model(x_test)
                loss_test = criterion(out_test, labels_test[:, 0:2])
                running_loss_test += loss_test.item()
                
        out_test = torch.cat((out_test, torch.unsqueeze(labels_test[:,2], dim=1)), dim=1)

        # Test error display
        true_test = Spher2Cart_1D(np.multiply(labels_test.cpu().data.numpy()[1], [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
        pred_test = Spher2Cart_1D(np.multiply(out_test.cpu().data.numpy()[1], [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
        print(true_test)
        print(pred_test)

        epoch_loss_train = running_loss_train * x_train.size(0) / len(training_names)
        epoch_loss_test = running_loss_test * x_test.size(0) / len(test_names)

        print('Train Loss: {:.6f} ---- Test Loss: {:.6f}'.format(epoch_loss_train, epoch_loss_test))
        if epoch % 5 == 0:
            if epoch_loss_test < best_error:
                torch.save(model.state_dict(), PATH)
                best_error = epoch_loss_test

main(names_FS, test_names, bs, num_epoch, y_FS, y_test, normalize=True)

Epoch 0/100
----------
[-8255.18942259  7711.56815611 -1009.16351357]
[-8277.83268109  7658.65198695 -1007.42637131]
[-8158.89281434  7668.78787744 -1009.55199919]
[-8212.78333504  7629.19432455 -1010.66489415]
Train Loss: 1.872207 ---- Test Loss: 2.408518
Epoch 1/100
----------
[-8255.18942259  7711.56815611 -1009.16351357]
[-8263.86391507  7684.67758597 -1008.09254961]
[-8158.89281434  7668.78787744 -1009.55199919]
[-8207.39440844  7641.34778799 -1011.05532802]
Train Loss: 1.362612 ---- Test Loss: 2.007727
Epoch 2/100
----------
[-8255.18942259  7711.56815611 -1009.16351357]
[-8250.33109672  7710.21173585 -1008.76366565]
[-8158.89281434  7668.78787744 -1009.55199919]
[-8200.63541456  7653.72176265 -1011.37028635]
Train Loss: 1.014777 ---- Test Loss: 1.759164
Epoch 3/100
----------
[-8255.18942259  7711.56815611 -1009.16351357]
[-8237.87645691  7734.39477264 -1009.42852875]
[-8158.89281434  7668.78787744 -1009.55199919]
[-8194.50971398  7664.99880918 -1011.6608769 ]
Train Loss: 0.8233

[-8158.89281434  7668.78787744 -1009.55199919]
[-8252.03111809  7583.27647583 -1010.45447607]
Train Loss: 0.375284 ---- Test Loss: 0.899627
Epoch 33/100
----------
[-8255.18942259  7711.56815611 -1009.16351357]
[-8245.89979097  7761.48996902 -1011.60862051]
[-8158.89281434  7668.78787744 -1009.55199919]
[-8254.64749281  7579.27769592 -1010.3843059 ]
Train Loss: 0.364044 ---- Test Loss: 0.888863
Epoch 34/100
----------
[-8255.18942259  7711.56815611 -1009.16351357]
[-8247.09941055  7758.02565731 -1011.47458657]
[-8158.89281434  7668.78787744 -1009.55199919]
[-8257.31843076  7575.31791323 -1010.32031657]
Train Loss: 0.354929 ---- Test Loss: 0.881388
Epoch 35/100
----------
[-8255.18942259  7711.56815611 -1009.16351357]
[-8248.39548403  7754.50902544 -1011.34371726]
[-8158.89281434  7668.78787744 -1009.55199919]
[-8259.83751886  7571.66544367 -1010.26513702]
Train Loss: 0.348715 ---- Test Loss: 0.876931
Epoch 36/100
----------
[-8255.18942259  7711.56815611 -1009.16351357]
[-8249.60035313

[-8158.89281434  7668.78787744 -1009.55199919]
[-8264.11628309  7562.56738516 -1009.995587  ]
Train Loss: 0.255878 ---- Test Loss: 0.778878
Epoch 65/100
----------
[-8255.18942259  7711.56815611 -1009.16351357]
[-8251.41463936  7742.6197087  -1010.8131832 ]
[-8158.89281434  7668.78787744 -1009.55199919]
[-8263.71521934  7562.91876807 -1009.99029959]
Train Loss: 0.253718 ---- Test Loss: 0.776900
Epoch 66/100
----------
[-8255.18942259  7711.56815611 -1009.16351357]
[-8251.16088293  7742.82396279 -1010.80913837]
[-8158.89281434  7668.78787744 -1009.55199919]
[-8263.43656523  7563.05379264 -1009.97998535]
Train Loss: 0.251556 ---- Test Loss: 0.774827
Epoch 67/100
----------
[-8255.18942259  7711.56815611 -1009.16351357]
[-8251.01679085  7742.78358579 -1010.79728372]
[-8158.89281434  7668.78787744 -1009.55199919]
[-8263.28055034  7562.97894154 -1009.9650525 ]
Train Loss: 0.249310 ---- Test Loss: 0.772657
Epoch 68/100
----------
[-8255.18942259  7711.56815611 -1009.16351357]
[-8250.98360792

[-8158.89281434  7668.78787744 -1009.55199919]
[-8257.77967795  7565.83258141 -1009.7730383 ]
Train Loss: 0.191435 ---- Test Loss: 0.716061
Epoch 97/100
----------
[-8255.18942259  7711.56815611 -1009.16351357]
[-8246.24311668  7742.30441541 -1010.45706015]
[-8158.89281434  7668.78787744 -1009.55199919]
[-8257.57918963  7566.09650052 -1009.77578534]
Train Loss: 0.189710 ---- Test Loss: 0.714497
Epoch 98/100
----------
[-8255.18942259  7711.56815611 -1009.16351357]
[-8246.05132084  7742.49787323 -1010.45639873]
[-8158.89281434  7668.78787744 -1009.55199919]
[-8257.40629393  7566.30135732 -1009.77676991]
Train Loss: 0.187995 ---- Test Loss: 0.712883
Epoch 99/100
----------
[-8255.18942259  7711.56815611 -1009.16351357]
[-8245.8809616   7742.62447685 -1010.45304585]
[-8158.89281434  7668.78787744 -1009.55199919]
[-8257.24087658  7566.450564   -1009.77486239]
Train Loss: 0.186285 ---- Test Loss: 0.711125


In [10]:
def Spher2Cart_2D(spherical):
    cartesian = np.zeros((len(spherical), 3))
    hypotenuse = np.multiply(np.cos(np.radians(spherical[:, 2])), spherical[:, 0])
    cartesian[:, 0] = np.multiply(np.cos(np.radians(spherical[:, 1])), hypotenuse)
    cartesian[:, 1] = np.multiply(-np.sin(np.radians(spherical[:, 1])), hypotenuse)
    cartesian[:, 2] = np.multiply(np.sin(np.radians(spherical[:, 2])), spherical[:, 0])
    return cartesian

# Testing: (range, az, el)
model.eval()
out_test_reg = np.zeros((len(y_test), 3))
labels_test_reg = np.zeros((len(y_test), 3))

for i in range(0, len(y_test) // bs):
    x_test, labels_test = Drawing_Batch(test_names, y_test, bs, i, f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/', True)
    x_test = x_test.to(device)
    labels_test = labels_test.cpu().data.numpy()
    labels_test_reg[bs * i: bs * i + bs] = (labels_test[:, 0:3])

    cur_test_reg = model(x_test)
    out_test_reg[bs * i: bs * i + bs, 0:2] = cur_test_reg.cpu().data.numpy()
    out_test_reg[bs * i: bs * i + bs, 2] = labels_test_reg[bs * i: bs * i + bs, 2]

# Calculate azimuth estimation error
azim_tot = 0
for i in range(len(out_test_reg)):
    azim_tot += np.linalg.norm(out_test_reg[i, 1] - labels_test_reg[i, 1])

azim_tot = azim_tot / len(out_test_reg)
print(f'Azimuth Estimation Error (deg) = {azim_tot}')

# Convert spherical coordinates to Cartesian coordinates
new_data = Spher2Cart_2D(np.multiply(out_test_reg, [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
new_labels_data = Spher2Cart_2D(np.multiply(labels_test_reg, [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)

# Calculate localization error
sum_tot = 0
for i in range(0, len(new_data) - (len(new_data) % bs), 1):
    sum_tot += np.linalg.norm(new_data[i, :] - new_labels_data[i, :])

sum_tot = sum_tot / (len(new_data) - (len(new_data) % bs))
print(f'Localization Error (m) = {sum_tot}')

Azimuth Estimation Error (deg) = 0.5053627958966188
Localization Error (m) = 47.23125310819853
